# Bibliotecas

In [110]:
# Import:
import pandas as pd
import pyodbc
from glob import glob
import fsspec
from urllib.parse import quote_plus
from sqlalchemy import create_engine
import locale
from datetime import datetime

# Lendo os arquivos (caminho)

In [111]:
# Digite o caminho e não se espeça de inserir a extensão dele no final da str
files = sorted(glob('C:\\Users\\alefc\\OneDrive\\Documentos\\Alura\\PBI\\01 - Challenge Alura log\\Bases\\*.csv'))
files

['C:\\Users\\alefc\\OneDrive\\Documentos\\Alura\\PBI\\01 - Challenge Alura log\\Bases\\Tabela - Estoque.csv',
 'C:\\Users\\alefc\\OneDrive\\Documentos\\Alura\\PBI\\01 - Challenge Alura log\\Bases\\Tabela - Pedidos.CSV',
 'C:\\Users\\alefc\\OneDrive\\Documentos\\Alura\\PBI\\01 - Challenge Alura log\\Bases\\Tabela - Produtos.csv',
 'C:\\Users\\alefc\\OneDrive\\Documentos\\Alura\\PBI\\01 - Challenge Alura log\\Bases\\Tabela - Veículos.csv']

In [112]:
dfEstoque = pd.read_csv(files[0])
dfProdutos = pd.read_csv(files[2])
dfVeiculos = pd.read_csv(files[3])
dfPedidos = pd.read_csv(files[1], sep=';')

In [113]:
dfVeiculos['ID veículos'] = dfVeiculos['ID veículos'].str.lstrip("VEH")
# Comando para criar o ID correto para os veículos

In [114]:
dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('.','')
dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('-','/')
dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('jan','01')
dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('fev','02')
dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('mar','03')
dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('abr','04')
dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('mai','05')
dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('jun','06')
dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('jul','07')
dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('ago','08')
dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('set','09')
dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('out','10')
dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('nov','11')
dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('dez','12')
dfEstoque['Data atualização'] = pd.to_datetime(dfEstoque['Data atualização'])

<ipython-input-114-ec4b16c4bbc7>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  dfEstoque['Data atualização'] = dfEstoque['Data atualização'].str.replace('.','')


In [115]:
dfPedidos['Data previsão'] = pd.to_datetime(dfPedidos['Data previsão'])

# SQL

In [116]:
# Conexão com BD:
parametros = (
    # Driver que será utilizado na conexão
    'DRIVER={SQL Server};'
    # IP ou nome do servidor.
    'SERVER=.;'
    # Banco que será utilizado.
    'DATABASE=Projetos;'
    # acesso pela credencial do windows
    'Trusted_Connection=yes')
    # Nome de usuário.
    # 'UID=login;'
    # Senha/Token.
    # 'PWD=senha')

url_db = quote_plus(parametros)

db = create_engine("mssql+pyodbc:///?odbc_connect=%s" % url_db)

conexao = db.connect()

In [117]:
dfEstoque.to_sql(
    name = 'dfEstoque',
    con = db,
    index = False,
    if_exists = 'replace'
)

dfProdutos.to_sql(
    name = 'dfProdutos',
    con = db,
    index = False,
    if_exists = 'replace'
)

dfVeiculos.to_sql(
    name = 'dfVeiculos',
    con = db,
    index = False,
    if_exists = 'replace'
)

dfPedidos.to_sql(
    name = 'dfPedidos',
    con = db,
    index = False,
    if_exists = 'replace'
)

In [118]:
dfSQL = pd.read_sql("select * from dfPedidos", db)

In [119]:
dfSQL.head()

,ID Pedido,ID Produto,Quantidade,ID Veículo,Status do pedido,Data da compra,Data de entrega,Data previsão,Latitude,Longitude,UF da entrega
0,1,32,3,19,Entregue,1/4/21 21:15,22/01/2021 21:15,2021-01-23 21:15:00,-22.19,-48.79,SP
1,2,50,9,42,Entregue,1/5/21 0:15,14/01/2021 00:15,2021-01-25 00:15:00,-22.19,-48.79,SP
2,3,39,3,19,Entregue,1/13/21 15:24,20/01/2021 15:24,2021-02-02 15:24:00,-20.51,-54.54,MS
3,4,39,1,16,Entregue,1/5/21 12:16,16/01/2021 12:16,2021-01-23 12:16:00,-6.60,-42.28,PI
4,5,19,7,14,Entregue,1/2/21 22:07,17/01/2021 22:07,2021-01-18 22:07:00,-22.25,-42.66,RJ
